<img src="../../../img/logo-bdc.png" align="right" width="64" />

# <span style="color:#336699">Image processing on images obtained through STAC</span>
<hr style="border:2px solid #0077b9;">

<div style="text-align: left;">
    <a href="https://nbviewer.jupyter.org/github/brazil-data-cube/code-gallery/blob/master/jupyter/Python/stac/stac-image-processing.ipynb"><img src="https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg" align="center"/></a>
</div>

<br/>

<div style="text-align: center;font-size: 90%;">
    Rennan Marujo<sup><a href="https://orcid.org/0000-0002-0082-9498"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>, Gilberto R. Queiroz<sup><a href="https://orcid.org/0000-0001-7534-0219"><i class="fab fa-lg fa-orcid" style="color: #a6ce39"></i></a></sup>
    <br/><br/>
    Earth Observation and Geoinformatics Division, National Institute for Space Research (INPE)
    <br/>
    Avenida dos Astronautas, 1758, Jardim da Granja, São José dos Campos, SP 12227-010, Brazil
    <br/><br/>
    Contact: <a href="mailto:brazildatacube@inpe.br">brazildatacube@inpe.br</a>
    <br/><br/>
    Last Update: March 22, 2021
</div>

<br/>

<div style="text-align: justify;  margin-left: 15%; margin-right: 15%;">
<b>Abstract.</b> This Jupyter Notebook describes how to search for CBERS-4 data products in <em>Brazil Data Cube</em>'s catalog through the STAC service. Then it shows how to use Python libraries to perform some image processing. It starts by computing the Normalized Difference Vegetation Index (NDVI) based on the red and near-infrared spectral bands. Next, it demonstrates a threshold analysis based on the computed NDVI. Lastly, it computes the NDVI difference for images from two diffrent dates.
</div>    

<br/>
<div style="text-align: justify;  margin-left: 25%; margin-right: 25%;font-size: 75%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>For an introduction to the SpatioTemporal Asset Catalog (STAC) with the <em>Brazil Data Cube</em> infrastructure, please, refer to the following Jupyter Notebook:</b>
    <div style="margin-left: 10px; margin-right: 10px">
    Zaglia, M.; Marujo, R.; Queiroz, G. R. <a href="./stac-introduction.ipynb" target="_blank">Introduction to the SpatioTemporal Asset Catalog (STAC)</a>.
    </div>
</div>

# STAC Client API
<hr style="border:1px solid #0077b9;">

For running the examples in this Jupyter Notebook you will need to install the [STAC client for Python](https://github.com/brazil-data-cube/stac.py). To install it from PyPI using `pip`, use the following command:

In [ ]:
#!pip install stac.py

In order to access the funcionalities of the client API, you should import the `stac` package, as follows:

In [ ]:
import stac

After that, you can check the installed `stac` package version:

In [ ]:
stac.__version__

Then, create a `STAC` object attached to the Brazil Data Cube' STAC service:

In [ ]:
service = stac.STAC('https://brazildatacube.dpi.inpe.br/stac/', access_token='change-me')

# Searching for CBERS-4 Images
<hr style="border:1px solid #0077b9;">

We are going to use the STAC `search` API to look for images from the data cube named `CB4_64_16D_STK`. This data cube is a temporal composite from CBERS-4/AWFI surface reflectance data. Let's define a search box with the following bounds: $x_{min} = -45.9$, $x_{max} = -45.4$, $y_{min} = -12.9$, $y_{max} = -12.6$. Besides that, the period of interest ranges from August 1st, 2018 to July 31st, 2019.

In [ ]:
bbox = (-45.9, -12.9, -45.4, -12.6)

In [ ]:
items = service.search({
    'collections': ['CB4_64_16D_STK-1'],
    'bbox': ','.join([str(coord) for coord in bbox]),
    'datetime': '2018-08-01/2019-07-31',
    'limit': 30
})

In [ ]:
items

The above query, should return 24 items for this particular data cube:

In [ ]:
len(items.features)

# Calculating the Normalized Difference Vegetation Index (NDVI)
<hr style="border:1px solid #0077b9;">

The normalized difference vegetation index (NDVI) is calculated using the **Red** and **Near Infrared** (NIR) spectral bands. It assesses whether or not the target being observed contains live green vegetation. It can be calculated through the following equation:

$$
NDVI = \frac{(NIR - RED)}{(NIR + RED)}
$$

<center><b>Equation 1</b> - NDVI.</center>

We are going to compute the NDVI just with images from the first item:

In [ ]:
item = items.features[0]

In order to know which band is available in an item, one can access its metadata:

In [ ]:
item

<div style="text-align: justify;  margin-left: 15%; margin-right: 15%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>Note:</b> As one can see, in the above metadata the Brazil Data Cube already provides the <em>NDVI</em> and <em>EVI</em> alongside with the spectral bands for the data cubes, besides the quality indicators (<em>CLEAROB</em>, <em>PROVENANCE</em>, <em>CMASK</em>, <em>TOTALOB</em>).
</div>

As one can see in the above cell, the `BAND15` correspond to the red wavelength and `BAND16` to the near-infrared.

Let's run an auxiliary script that prepares a function named `read_raster` for reading the `read` and `nir` bands:

In [ ]:
%run utils.py

The above code will add `read_raster` to the scope. This function allows to retrieve part of an image according to a rectangle specified in `EPSG:4326`:

In [ ]:
red = read_raster(item, 'BAND15', bbox)

In [ ]:
red

In [ ]:
nir = read_raster(item, 'BAND16', bbox)

In [ ]:
nir

Let's take a look at the retrieved data:

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.imshow(red, cmap='gray');

In [ ]:
plt.imshow(nir, cmap='gray');

Finally, let's compute the NDVI:

In [ ]:
ndvi = (nir - red)/(nir + red)
ndvi

Use the Matplotlib to visualize the result array:

In [ ]:
plt.imshow(ndvi, cmap='gray');

We can also plot using other colormaps:

In [ ]:
plt.imshow(ndvi, cmap='jet');

For more colormaps check https://matplotlib.org/stable/tutorials/colors/colormaps.html

# Thresholding images
<hr style="border:1px solid #0077b9;">

One of the simplest approaches to separate different values in images is thresholding. This process consists in labeling data based on fixed values.

Let's try to separate our data into groups according to their NDVI values.
But first let's see how the image histogram behaves:

In [ ]:
plt.title("NDVI Histogram")
plt.hist(ndvi)
plt.show()

Supposing we can separate the `ndvi` image with threshold, we would assume for this specific case that:
* all pixels with values below 0.2 are dark pixels;
* all pixels above 0.45 are areas containing a good portion of vegetation.
* all pixels with values from 0.2 to 0.45 are areas with few vegetation;

We can perform this thresholding by selecting in the ndvi matrix all values belonging to a given range and assigning a common integer value. We assume the following integer values:
* `1`: dark pixels;
* `2`: vigorous vegetation;
* `3`: weak vegetation.

Let's first create a copy of the original ndvi matrix:

In [ ]:
labeled_img = ndvi.copy()

Now, we can use the new copy of the ndvi array and assign the values according to each range of values:

In [ ]:
labeled_img[ndvi < 0.2] = 1 # < 0.2
labeled_img[ndvi >= 0.2] = 3 # 0.2 - 0.45
labeled_img[ndvi >= 0.45] = 2 # >= 0.45
labeled_img

Finally, let's see the `ndvi` image separated into those labels:

In [ ]:
plt.rcParams['figure.figsize'] = [30, 20] #Change plot size
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(ndvi, cmap='gray')
ax2.imshow(labeled_img, cmap='brg');

# Calculating Image Difference
<hr style="border:1px solid #0077b9;">

Now let's suppose we want to compare the NDVI for images from two different dates and same location, for instance to verify the areas where crops have grown and areas that loss vegetation.

For this computation we are going to use the NDVI bands provided in the data cube, and we will select two items (with the same location but with different dates) using STAC.

The first image comprises pixels from July 28th, 2019 to August 12th, 2019 (`2019-07-28_2019-08-12`):

In [ ]:
first_item = items.features[0]

In [ ]:
first_item

In [ ]:
ndvi_first_image = read_raster(first_item, 'NDVI', bbox)

The other selected image comprises pixels from January 01st, 2019 to January 16th, 2019 (`2019-01-01_2019-01-16`) - i.e., six months before the first one selected:

In [ ]:
second_item = items.features[13]

In [ ]:
second_item

In [ ]:
ndvi_second_image = read_raster(second_item, 'NDVI', bbox)

<div style="text-align: justify;  margin-left: 15%; margin-right: 15%; border-style: solid; border-color: #0077b9; border-width: 1px; padding: 5px;">
    <b>Note:</b> NDVI bands precomputed by BDC ranges from <em>-10000</em> to <em>10000</em>, instead of <em>-1</em> to <em>1</em>, as can be seen in the item metadata. This is due to the lower volume required to store files that use 16-bit integer values rather than 32-bit float.
</div>

Considering that these images are from an agricultural area and that crops are normally planted near August (first observation), in a six months, previous or after the first observation, it is expected to find crops, which will imply in greater NDVI values (more vigorous vegetation). This will cause NDVI band to present brighter values on these areas. Using the `gray` colormap, high value NDVI pixels will be more similar to white, while low value NDVI pixels will be closer to the black color.
Based on that, let's visually compare both NDVI images:

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(ndvi_first_image, cmap='gray')
ax2.imshow(ndvi_second_image, cmap='gray');

Assuming we want to see what have grown and what was loss, let's subtract the most recent image from the oldest one and plot it:

In [ ]:
ndvi_diff = ndvi_first_image - ndvi_second_image

In [ ]:
plt.rcParams['figure.figsize'] = [10, 5]
plt.imshow(ndvi_diff, cmap='jet');

As can be seen in the the NDVI difference plot, the main changes on pixel values were found in agriculture areas, which was expected due to changes in crops.
The blue values indicate negative values, while red values are positive. This means that for the blue areas there was a loss of vegetation, as a decreasing result on the NDVI value, meaning that crops were harvest. Meanwhile, on the red areas, the NDVI value increased as a result of the more vigorous vegetation on the recent date.

# References
<hr style="border:1px solid #0077b9;">

- [Brazil Data Cube Python Client Library for STAC Service - GitHub Repository](https://github.com/brazil-data-cube/stac.py)

- [Spatio Temporal Asset Catalog Specification](https://stacspec.org/)

# See also the following Jupyter Notebooks
<hr style="border:1px solid #0077b9;">

* [Introduction to the SpatioTemporal Asset Catalog (STAC)](./stac-introduction.ipynb)